# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import numpy as np
import pandas as pd
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [2]:
# Константы

BUDGET = 10e9 #Бюджет на разработку скважин в регионе
ONE_THOUSAND_BARREL_PRICE = 450_000 #Доход с каждой единицы продукта составляет 450 тыс. рублей, 
#поскольку объём указан в тысячах баррелей.


POINTS = 500 #При разведке региона исследуют 500 точек
BEST_POINTS = 200 #выбирают 200 лучших для разработки

In [3]:
try:
    data1 = pd.read_csv('/datasets/geo_data_0.csv')
    data2 = pd.read_csv('/datasets/geo_data_1.csv')
    data3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data1 = pd.read_csv('./datasets/geo_data_0.csv')
    data2 = pd.read_csv('./datasets/geo_data_1.csv')
    data3 = pd.read_csv('./datasets/geo_data_2.csv')

In [4]:
# check

print(data1.head(5))
print(data2.head(5))
print(data3.head(5))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [5]:
# check

print(data1.info())
print(data2.info())
print(data3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [6]:
# check

print(data1.describe())
print(data2.describe())
print(data3.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

In [7]:
# Проверим дубликаты

print(data1['id'].duplicated().sum())
print(data2['id'].duplicated().sum())
print(data3['id'].duplicated().sum())

10
4
4


In [8]:
# Проверим корреляцию признаков

data1.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [9]:
data2.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Самая большая корреляция во втором датасете между f2 и product

In [10]:
data3.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [11]:
# Удаляем неважные признаки

data1 = data1.drop(columns=['id'])
data2 = data2.drop(columns=['id'])
data3 = data3.drop(columns=['id'])

### Вывод

- Изучила датафреймы по 3м регионам
- Проверены дубликаты и корреляция по 3м регионам
- Удалена лишняя метрика id
- Масштабированы данные

## Обучение и проверка модели

### Обучаем модель по 1-му региону


In [12]:
# Делим на выборки

target1 = data1['product']
features1 = data1.drop(['product'] , axis=1)

features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    features1, 
    target1, 
    train_size=0.75, 
    random_state=12345, 
)

In [13]:
numeric = ['f0', 'f1', 'f2']

In [14]:
# Выполняем масштабирование

warnings.filterwarnings("ignore")

scaler = StandardScaler()

features_train1[numeric] = scaler.fit_transform(features_train1[numeric])
features_valid1 = scaler.transform(features_valid1)


In [15]:
# check

features_train1.shape

(75000, 3)

In [16]:
features_valid1.shape

(25000, 3)

In [17]:
target_train1.shape

(75000,)

In [18]:
target_valid1.shape

(25000,)

In [19]:
# Обучаем линейную регрессию

model = LinearRegression()
model.fit(features_train1, target_train1)
predictions_valid1 = model.predict(features_valid1)

data1_pred_mean = predictions_valid1.mean()

result = (mean_squared_error(target_valid1, predictions_valid1))**0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result)
print('Cредний запас предсказанного сырья:', data1_pred_mean)

RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Cредний запас предсказанного сырья: 92.59256778438035


### Обучаем модель по 2-му региону

In [20]:
# Делим на выборки

target2 = data2['product']
features2 = data2.drop(['product'] , axis=1)

features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    features2, 
    target2, 
    train_size=0.75, 
    random_state=12345, 
)

In [21]:
# Выполняем масштабирование

numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train2[numeric])

features_train2[numeric] = scaler.transform(features_train2[numeric])
features_valid2 = scaler.transform(features_valid2)

pd.options.mode.chained_assignment = None

In [22]:
# check

print(features_train2.shape)
print(features_valid2.shape)
print(target_train2.shape)
print(target_valid2.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [23]:
# Обучаем линейную регрессию

model = LinearRegression()
model.fit(features_train2, target_train2)
predictions_valid2 = model.predict(features_valid2)

data2_pred_mean = predictions_valid2.mean()

result = (mean_squared_error(target_valid2, predictions_valid2))**0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result)
print('Cредний запас предсказанного сырья:', data2_pred_mean)

RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617
Cредний запас предсказанного сырья: 68.728546895446


### Обучаем модель по 3-му региону

In [24]:
# Делим на выборки

target3 = data3['product']
features3 = data3.drop(['product'] , axis=1)

features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    features3, 
    target3, 
    train_size=0.75, 
    random_state=12345, 
)

In [25]:
# Выполняем масштабирование

numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train3[numeric])

features_train3[numeric] = scaler.transform(features_train3[numeric])
features_valid3 = scaler.transform(features_valid3)

pd.options.mode.chained_assignment = None

In [26]:
# check

print(features_train3.shape)
print(features_valid3.shape)
print(target_train3.shape)
print(target_valid3.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)


In [27]:
# Обучаем линейную регрессию

model = LinearRegression()
model.fit(features_train3, target_train3)
predictions_valid3 = model.predict(features_valid3)

data3_pred_mean = predictions_valid3.mean()

result = (mean_squared_error(target_valid3, predictions_valid3))**0.5
print("RMSE модели линейной регрессии на валидационной выборке:", result)
print('Cредний запас предсказанного сырья:', data3_pred_mean)

RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Cредний запас предсказанного сырья: 94.96504596800489


### Вывод

- 1 и 3 датафреймы показали прмерно похожие результаты RMSE и запасов сырья.
- 2й регион показывает лучшую цифру RMSE, но сильно проседает по запасам сырья.

## Подготовка к расчёту прибыли

### Все ключевые значения для расчётов сохраним в отдельных переменных.

###  Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе. 

In [28]:
best_profit = BUDGET / BEST_POINTS

In [29]:
best_profit #лучшая прибыль

50000000.0

In [30]:
mean_volume_oil_point = best_profit / ONE_THOUSAND_BARREL_PRICE

In [31]:
mean_volume_oil_point #средний объем нефти на одну точку в тыс. баррелей

111.11111111111111

In [32]:
mean_volume_oil_region = BUDGET / ONE_THOUSAND_BARREL_PRICE

In [33]:
mean_volume_oil_region ##средний объем нефти на регион в тыс. баррелей

22222.222222222223

In [34]:
# Текущие запасы

print('Cредний запас 1 региона:', data1_pred_mean)
print('Cредний запас 2 региона:', data2_pred_mean)
print('Cредний запас 3 региона:', data3_pred_mean)

Cредний запас 1 региона: 92.59256778438035
Cредний запас 2 региона: 68.728546895446
Cредний запас 3 региона: 94.96504596800489


### Вывод

Требуемое количество добычи нефти сильно превышает текущие запасы, разработка новой скважины будет убыточной

## Расчёт прибыли и рисков 

### Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [35]:
def calculation_profit_and_risks(target, predictions, points):
    preds_sorted = predictions.sort_values(ascending=False)
    selected = target[preds_sorted.index][:points]
    return ONE_THOUSAND_BARREL_PRICE * selected.sum() - BUDGET

In [36]:
# применяем функцию для 500 точек 1го региона

calculation_profit_and_risks(target_valid1, pd.Series(predictions_valid1), BEST_POINTS)

-1922149005.722169

In [37]:
# применяем функцию для 500 точек 3го региона

calculation_profit_and_risks(target_valid3, pd.Series(predictions_valid3), BEST_POINTS)

-1259258892.6866302

In [43]:
# ячейка ревьюера

calculation_profit_and_risks(target_valid3.reset_index(drop=True), pd.Series(predictions_valid3), BEST_POINTS)

2710349963.5998325

### Функция расчета рисков и прибыли для каждого региона с техникой Bootstrap с 1000 выборок

In [38]:
def bootstrap_func(target, predictions):
    state = np.random.RandomState(12345)
    
    values = []
    for i in range(1000):
        target = target.reset_index(drop=True)
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]  

        res = calculation_profit_and_risks(target_subsample, probs_subsample, 200)
        values.append(res)

    values = pd.Series(values)
    lower = values.quantile(.025)
    higher = values.quantile(.975)

    mean = values.mean() 
    print("Средняя прибыль:", mean)
    print("5%-квантиль:", lower)
    print("95%-квантиль:", higher)
    print('Риск убытков = {:.2%} '.format((values<0).mean()))

In [39]:
#ячейка ревьюера


'Риск убытков = {:.2%} '.format((pd.Series([-2, -1, 4, 5])<0).mean())

'Риск убытков = 50.00% '

In [40]:
bootstrap_func(target_valid1, pd.Series(predictions_valid1))

Средняя прибыль: 425938526.91059244
5%-квантиль: -102090094.83793654
95%-квантиль: 947976353.3583689
Риск убытков = 6.00% 


In [41]:
bootstrap_func(target_valid2, pd.Series(predictions_valid2))

Средняя прибыль: 515222773.4432899
5%-квантиль: 68873225.37050176
95%-квантиль: 931547591.2570494
Риск убытков = 1.00% 


In [42]:
bootstrap_func(target_valid3, pd.Series(predictions_valid3))

Средняя прибыль: 435008362.7827556
5%-квантиль: -128880547.32978901
95%-квантиль: 969706954.1802661
Риск убытков = 6.40% 


### Вывод

Сравнивая 3 региона, для разработки больше всего подходит 2й регион. Там средняя прибыль выше, чем у остальных и риск убытков самый низкий.

## Общий вывод

**Подготовка данных**

- Были изучены датафреймы по 3м регионам
- Удалена лишняя метрика id
- Масштабированы данные

**Обучение модели**
- Данные по каждому из регионов были разбиты на тренировочную и валидационную выборки
- Построена модель линейной регрессии 
- Рассчитаны RMSE и средний запас предсказанного сырья для каждого региона

**Рассчитан объем сырья, необходимый для безубыточной разработки скважины**


**Рассчитана прибыль и риски**
- По рассчетам выбран лучший регион для разрабоки новой скважины - 2й. У него самый маленкий риск убытков и с прибылью все ок.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован